Slack-Gmail Summariser chatbot

In this tutorial/documentation, I will show how to build a Slackbot that integrates with your Gmail to summarize the emails received in the last hour. This simple app can be expanded by providing better summaries, ignoring certain types of Emails, and making it more bespoke to you.


*STEP 0*

In order to run all the following code and get access to the GUI, we first need to install Docker Hub here:https://docs.docker.com/get-docker/. Once this is installed on your computer, you can then run the following command on your terminal.


In [ ]:
docker run --name mindsdb_container -p 47334:47334 -p 47335:47335 mindsdb/mindsdb


With this, you will locally install the MindsDB image and run the container on the designated ports (47334 & 47335). If these ports are occupied, you can modify the port to your liking by changing the first number before the ":" If you change the ports, the code below will need some slight modification.


*STEP 1*


To begin, we must download the SDK so that we can interface with Minds DB. This tool allows us to give our databases ML and LLM capabilities and easily add or create new databases.

to read more about the SDKs they offer aside from Python click here:
https://docs.mindsdb.com/sdks/overview


In [ ]:
pip install mindsdb_sdk


It is preferable that the install is done Through the terminal as it can avoid certain issues later on.
to do it run the same piece of code but in the terminal if there is any issue try this variation.

In [ ]:
py -m pip install mindsdb_sdk


*STEP 2*

Connecting with MindsDB is fairly simple. For a Full guide, see https://docs.mindsdb.com/sdks/python/connect. However, here, we will go with a quick explanation of what the code is doing. 



In [1]:
import mindsdb_sdk

# connects to the specified host and port
server = mindsdb_sdk.connect('http://127.0.0.1:47334')


The first line of code simply imports the MindsDB library so that we can use its functions. We then call upon the connect function and save it in the server variable. The last line of code shows that it can be connected to a specified port if you so wish. 



*STEP 3*

We can see our current databases now that we have successfully connected with the minds db servers. This should be blank apart from the default database. Since we will be using an integration, it is recommended that you have the GUI running on your preferred browser. You can run it by doing "CTR+click" on the HTTP link above 



In [ ]:
server.list_databases()

In [ ]:
mindsdb = server.get_project('mindsdb')
data_handlers = mindsdb.query('SHOW HANDLERS WHERE type = \'data\'')
print(data_handlers.fetch())


Here, we can check what data tools we have and which we can install. For this project, we need to install the Gmail integration. To do so, we can use the GUI.

    -Click on the "add" button on the top left side of the screen 

    -Then select "new data source" 

    -This will take you to a page with all data source integrations; look it up and select "Gmail"

    -Give the data source a name you want 

    -add your credentials JSON (to get this file, follow this tutorial: https://docs.mindsdb.com/integrations/app-integrations/gmail)

    -Once added, click on "test connection," and you will be prompted with a Google Oauhth screen.

    -Once this is completed, you should be able to click "save and continue." Click this button, and your database will be created.


In [ ]:
gmail_access = server.create_database(
    engine = "gmail",
    name = "gmail_access",
    connection_args = {
        "credentials_file": "<YOUR CREDENTIALS FILE DIRECTORY>",
        "scopes": ['https://.../gmail.compose', 'https://.../gmail.readonly']
    }
)


In [ ]:
gmail_access = server.get_database('gmail_access')


Now that our database is created, we can run this code to check it. Feel free to run any Queries to check the structure in which our Emails are extracted. This structure is important to understand how it is that we will process them later on. If you have any questions on how to run the Queries, you can use the GUI's SQL editor or run the Python code:




In [ ]:
query_test=gmail_access.query('<your test SQL QUERY>')
query_test.fetch()


*STEP 4*

Now that you have explored the Gmail_access database, you can proceed to create our ML/LLM engine to Process Emails as needed. Here, you can use any ML engine you want, from common ones like Openai or Meta to more niche ones like Huggingface. So feel free to modify this code to suit your needs.



The first step is to create an engine from which we can then create our model. This next piece of code shows how. The first text in" refers to the name that our engine will have, and the second is the provider, in this case, OpenAI. The connection data argument just references the API key we need to run our models.



In [ ]:
server.ml_engines.create(
       'openai_engine',
       'openai',
       connection_data={'openai_api_key': '<YOUR API KEY>'}
   )


once the engine is created we need to create a proyect in wich to house all of our models will be stored. we do this by running the folowing command.

In [ ]:
project = server.create_project('summariser_bot')


Now we can create our model; to do this, we need to specify the arguments we need. The name, engine, what it will predict, and the prompt on which it will run things. These parameters change depending on the engine you use and the purpose you give to the model. Check the documentation for your use case. (https://docs.mindsdb.com/use-cases/overview)



In [ ]:
text_summarization_model = project.models.create (
      name='text_summarization_model',
      engine='openai_engine', # alternatively: engine=server.ml_engines.openai
      predict='snippet',
      prompt_template='provide an informative summary of the text text:{{body}} using full sentences in 100 words or less, the information should be presented in bulletpoints, there should be 7 or less bulletpoints, ignore talking about promotional material, be very selective about only mentioning critical information',
      #model_name='gpt3.5'
)


Now to check everything was created as we wanted run the following code:

In [ ]:
text_summarization_model = project.models.get('text_summarization_model')


since what we want to predict are snippets of the body in a certain format to get a batch of predictions we can run the following SQL query 

In [ ]:
query = project.query('SELECT input.body, output.snippet FROM gmail_access.emails AS input JOIN text_summarization_model AS output LIMIT 1;')
snippets = query.fetch()
Print(snippets)

Unfortunately, if you want to make any changes to the model prompt or its engine you'll need to delete them and re-create them. to do this, I recommend using the GUI as it is less prone to errors. check the documentation for what Query to run (https://docs.mindsdb.com/mindsdb_sql/overview)



*STEP 5*

Now that we have our sumarization for our emails we need to conect it to Slack or any other chat so taht we can get our TLDR bot to tell us what is happening in our Emails.

In order to connect to Slack, we create a new database here rather than extract data from it. Which is also possible, we will put data on it. to do this, we can run the following code with the parameters needed for the Slack integration. Like the Gmail integration, you will need to create a token with the proper scope and a bot that Slack can add to the channel. To do this, follow the tutorial here(https://docs.mindsdb.com/integrations/app-integrations/slack)



In [ ]:
slack_conn = server.create_database(
    engine = "slack",
    name = "slack_conn",
    connection_args = {
      "token": "<YOUR SLACKBOT API TOKEN>"
    }
)

once created you can run the list databases command to ensure it was created sucessfuly. you should also at this point create a alsck channel for your application and add the bot on the slack channel.

In [ ]:
server.list_databases()

*STEP 6*

Now, we can run a query to insert text into a designated channel. To do this, we follow the query structure of the documentation above. It is important to add the "AS channel" and "AS text;" otherwise, the formatting may be wrong. Once this query is run, we will see an update on the Slack channel every time we re-run it. By default, it brings out the latest emails. I personally put a limit as to how many it should bring me.



In [ ]:
query2 = slack_conn.query('INSERT INTO slack_conn.channels (channel, text) SELECT "mindsdb-test" AS channel, output.snippet AS text FROM gmail_access.emails AS input JOIN summariser_bot.text_summarization_model AS output LIMIT 5')
query2.fetch()



Check slack to see everything is going well so we can do the next step automating the application.

*STEP 7*

In order to automate this application, we need to take advantage of the jobs that MIndsDB provides. In practice, these jobs run a query over a designated amount of time until they are stopped. We will create two Jobs: one to refresh our predictions and one to refresh the Slack message query.



In [ ]:
gmail_refresh = project.create_job(
    'gmail_refresh',
    'SELECT input.body, output.snippet FROM gmail_access.emails AS input JOIN text_summarization_model AS output LIMIT 5;',
    repeat_str = '1 hour'
)

In [ ]:
slack_refresh = project.create_job(
    'slack_refresh',
    'INSERT INTO slack_conn.channels (channel, text) SELECT "mindsdb-test" AS channel, output.snippet AS text FROM gmail_access.emails AS input JOIN summariser_bot.text_summarization_model AS output LIMIT 5;',
    repeat_str = '1 hour'
)


As we can see, both jobs are very similar. The difference is the query they are running, which is the same query as above. Something important to mention is that the jobs will only run as long as there is a connection to the MindsDB servers. As soon as you close up the container in which everything is running, the jobs will stop.



*CONCLUSION*

We have created a Gmail summary integrated with Slack to give us our summaries via Slack. A couple of issues may arise, and the most relevant is the gmail_access database losing clearance. If this happens, the whole database must be dropped and re-done. While there is code to make it over Python, I could not get it to work as it did not detect my credentials JSON as valid. Another issue may arise when dropping/deleting jobs, databases or models as they may be deleted, but errors will arise saying the database already exists. If this happens, you must reset the container from the docker hub.

